# "Understanding and Predicting Power Outage Patterns in the United States"

**Name(s)**: xiulin chen

**Website Link**: https://isabelccc.github.io/eecs398_project.github.io/

In [63]:
import pandas as pd

df = pd.read_excel("outage.xlsx", header=5)
print(df.columns)
df.head()

print("Rows:", df.shape[0])
print("Columns:", df.columns.tolist())

Index(['variables', 'OBS', 'YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE',
       'NERC.REGION', 'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY',
       'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE',
       'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL',
       'HURRICANE.NAMES', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW',
       'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE',
       'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES', 'TOTAL.SALES',
       'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.CUSTOMERS',
       'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS', 'RES.CUST.PCT',
       'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE', 'PC.REALGSP.USA',
       'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP', 'TOTAL.REALGSP',
       'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'POPULATION', 'POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL',
       'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WAT

## Step 1:Introduction and Question Identification
Our project leverages a comprehensive dataset detailing major power outage events across the United States, spanning from the year 2000 to 2016. Each row in the dataset represents a single outage incident, enriched with contextual features such as climate conditions, regional infrastructure metrics, population data, and outage metadata. The dataset originates from U.S. government and energy infrastructure reports.

Total Rows: ~1535

	•	Key columns for analysis:
	•	YEAR, MONTH: When the outage happened
	•	U.S._STATE, CLIMATE.REGION: Where it occurred
	•	CLIMATE.CATEGORY, ANOMALY.LEVEL: Climate conditions
	•	CUSTOMERS.AFFECTED: Impact
	•	OUTAGE.START, OUTAGE.RESTORATION: Timing
	•	CAUSE.CATEGORY: What caused the outage



## Step 2: Data Cleaning and Exploratory Data Analysis


In [61]:
df['U.S._STATE'].unique()
df_valid = df[df["U.S._STATE"].notna()]

# Make a copy to avoid SettingWithCopyWarning
df_valid = df_valid.copy()

# Convert date and time columns to string explicitly
df_valid["OUTAGE.START.DATE"] = pd.to_datetime(df_valid["OUTAGE.START.DATE"], errors='coerce').dt.date.astype(str)
df_valid["OUTAGE.START.TIME"] = df_valid["OUTAGE.START.TIME"].astype(str)

df_valid["OUTAGE.RESTORATION.DATE"] = pd.to_datetime(df_valid["OUTAGE.RESTORATION.DATE"], errors='coerce').dt.date.astype(str)
df_valid["OUTAGE.RESTORATION.TIME"] = df_valid["OUTAGE.RESTORATION.TIME"].astype(str)

# Combine date and time strings and convert to datetime
df_valid["OUTAGE.START"] = pd.to_datetime(
    df_valid["OUTAGE.START.DATE"] + " " + df_valid["OUTAGE.START.TIME"],
    format="%Y-%m-%d %H:%M:%S", errors="coerce"
)

df_valid["OUTAGE.RESTORATION"] = pd.to_datetime(
    df_valid["OUTAGE.RESTORATION.DATE"] + " " + df_valid["OUTAGE.RESTORATION.TIME"],
    format="%Y-%m-%d %H:%M:%S", errors="coerce"
)

# Calculate outage duration in hours
df_valid["DURATION_HOURS"] = (df_valid["OUTAGE.RESTORATION"] - df_valid["OUTAGE.START"]).dt.total_seconds() / 3600

# Drop invalid rows
df_valid = df_valid.dropna(subset=["OUTAGE.START", "OUTAGE.RESTORATION", "DURATION_HOURS"])

# Ensure numeric columns are cast correctly:
df_valid["CUSTOMERS.AFFECTED"] = pd.to_numeric(df_valid["CUSTOMERS.AFFECTED"], errors="coerce")
df_valid["ANOMALY.LEVEL"] = pd.to_numeric(df_valid["ANOMALY.LEVEL"], errors="coerce")


# Remove duration and customer outliers
duration_threshold = df_valid["DURATION_HOURS"].quantile(0.999)
df_valid = df_valid[
    (df_valid["DURATION_HOURS"] <= duration_threshold) &
    (df_valid["CUSTOMERS.AFFECTED"] < 1e7)
]

# Show result
print(df_valid[["U.S._STATE", "OUTAGE.START", "OUTAGE.RESTORATION", "DURATION_HOURS"]].head())
df_valid.shape[0]

  U.S._STATE        OUTAGE.START  OUTAGE.RESTORATION  DURATION_HOURS
1  Minnesota 2011-07-01 17:00:00 2011-07-03 20:00:00            51.0
3  Minnesota 2010-10-26 20:00:00 2010-10-28 22:00:00            50.0
4  Minnesota 2012-06-19 04:30:00 2012-06-20 23:00:00            42.5
5  Minnesota 2015-07-18 02:00:00 2015-07-19 07:00:00            29.0
6  Minnesota 2010-11-13 15:00:00 2010-11-14 22:00:00            31.0


1056

In [62]:
relevant_columns = [
    'YEAR', 'MONTH', 'U.S._STATE', 'CLIMATE.REGION', 'ANOMALY.LEVEL',
    'CLIMATE.CATEGORY', 'CUSTOMERS.AFFECTED', 'POPDEN_URBAN', 'POPDEN_RURAL',
    'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WATER_TOT', 'PCT_WATER_INLAND',
    'OUTAGE.START', 'OUTAGE.RESTORATION', 'DURATION_HOURS'
]

df_filtered = df_valid[relevant_columns].copy()
df_filtered.head()

,YEAR,MONTH,U.S._STATE,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CUSTOMERS.AFFECTED,POPDEN_URBAN,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START,OUTAGE.RESTORATION,DURATION_HOURS
1,2011.0,7.0,Minnesota,East North Central,-0.3,normal,70000.0,2279,18.2,2.14,0.6,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00,51.0
3,2010.0,10.0,Minnesota,East North Central,-1.5,cold,70000.0,2279,18.2,2.14,0.6,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00,50.0
4,2012.0,6.0,Minnesota,East North Central,-0.1,normal,68200.0,2279,18.2,2.14,0.6,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00,42.5
5,2015.0,7.0,Minnesota,East North Central,1.2,warm,250000.0,2279,18.2,2.14,0.6,91.592666,8.407334,5.478743,2015-07-18 02:00:00,2015-07-19 07:00:00,29.0
6,2010.0,11.0,Minnesota,East North Central,-1.4,cold,60000.0,2279,18.2,2.14,0.6,91.592666,8.407334,5.478743,2010-11-13 15:00:00,2010-11-14 22:00:00,31.0


In [55]:
import plotly.figure_factory as ff
import plotly.express as px

fig = px.box(df_valid, y="DURATION_HOURS", title="Outage Duration Distribution (Box Plot)")
fig.write_html("univariate_duration_box.html")

# Histogram of CUSTOMERS.AFFECTED
fig2 = px.histogram(df_valid, x="CUSTOMERS.AFFECTED", nbins=50, title="Distribution of Customers Affected")
fig2.write_html("univariate_customers.html")

*Univarite analysis :duration hours*

This box plot illustrates the distribution of outage durations in hours. The median outage duration is approximately 21 hours, with most outages lasting under 60 hours. However, there are several extreme events exceeding 140 hours, which are visualized as outliers. These long outages may be due to severe weather or infrastructure failures and are rare but impactful. This insight supports our focus on duration as a key indicator of severity.

*Univariate Analysis: Customers Affected (Histogram)*

This histogram displays the distribution of the number of customers affected by each outage. The distribution is highly right-skewed — most outages affect fewer than 250,000 customers, while a few extreme cases impact over 1 million. These rare but massive events are important for understanding system vulnerability and planning mitigation efforts. The presence of such a long tail highlights the need for careful handling of this variable during model training, potentially via transformation or robust scaling.

In [42]:
import folium
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
state_coords = {
    'Alabama': [32.806671, -86.791130],
    'Alaska': [61.370716, -152.404419],
    'Arizona': [33.729759, -111.431221],
    'Arkansas': [34.969704, -92.373123],
    'California': [36.116203, -119.681564],
    'Colorado': [39.059811, -105.311104],
    'Connecticut': [41.597782, -72.755371],
    'Delaware': [39.318523, -75.507141],
    'District of Columbia': [38.897438, -77.026817],
    'Florida': [27.766279, -81.686783],
    'Georgia': [33.040619, -83.643074],
    'Hawaii': [21.094318, -157.498337],
    'Idaho': [44.240459, -114.478828],
    'Illinois': [40.349457, -88.986137],
    'Indiana': [39.849426, -86.258278],
    'Iowa': [42.011539, -93.210526],
    'Kansas': [38.526600, -96.726486],
    'Kentucky': [37.668140, -84.670067],
    'Louisiana': [31.169546, -91.867805],
    'Maine': [44.693947, -69.381927],
    'Maryland': [39.063946, -76.802101],
    'Massachusetts': [42.230171, -71.530106],
    'Michigan': [43.326618, -84.536095],
    'Minnesota': [45.694454, -93.900192],
    'Mississippi': [32.741646, -89.678696],
    'Missouri': [38.456085, -92.288368],
    'Montana': [46.921925, -110.454353],
    'Nebraska': [41.125370, -98.268082],
    'Nevada': [38.313515, -117.055374],
    'New Hampshire': [43.452492, -71.563896],
    'New Jersey': [40.298904, -74.521011],
    'New Mexico': [34.840515, -106.248482],
    'New York': [42.165726, -74.948051],
    'North Carolina': [35.630066, -79.806419],
    'North Dakota': [47.528912, -99.784012],
    'Ohio': [40.388783, -82.764915],
    'Oklahoma': [35.565342, -96.928917],
    'Oregon': [44.572021, -122.070938],
    'Pennsylvania': [40.590752, -77.209755],
    'Rhode Island': [41.680893, -71.511780],
    'South Carolina': [33.856892, -80.945007],
    'South Dakota': [44.299782, -99.438828],
    'Tennessee': [35.747845, -86.692345],
    'Texas': [31.054487, -97.563461],
    'Utah': [40.150032, -111.862434],
    'Vermont': [44.045876, -72.710686],
    'Virginia': [37.769337, -78.169968],
    'Washington': [47.400902, -121.490494],
    'West Virginia': [38.491226, -80.954570],
    'Wisconsin': [44.268543, -89.616508],
    'Wyoming': [42.755966, -107.302490]
}


# Create a copy to avoid SettingWithCopyWarning
df_valid = df_valid.copy()

# Add latitude and longitude safely
df_valid["LAT"] = df_valid["U.S._STATE"].map(lambda x: state_coords.get(x, [None, None])[0])
df_valid["LON"] = df_valid["U.S._STATE"].map(lambda x: state_coords.get(x, [None, None])[1])

# Drop rows with missing coordinates
df_valid.dropna(subset=["LAT", "LON"], inplace=True)

# Create GeoDataFrame
gdf = gpd.GeoDataFrame(df_valid, geometry=gpd.points_from_xy(df_valid["LON"], df_valid["LAT"]), crs="EPSG:4326")

In [ ]:
import plotly.express as px

duration_by_cause = df_valid.groupby('CAUSE.CATEGORY')['DURATION_HOURS'].mean().sort_values(ascending=False).reset_index()

fig2 = px.bar(
    duration_by_cause,
    x='CAUSE.CATEGORY',
    y='DURATION_HOURS',
    title='Average Outage Duration by Cause',
    labels={'CAUSE.CATEGORY': 'Cause of Outage', 'DURATION_HOURS': 'Average Duration (Hours)'},
    color='CAUSE.CATEGORY'
)
fig2.update_layout(xaxis_title="Cause of Outage", yaxis_title="Average Duration (Hours)")

# Save the graph to an HTML file
fig2.write_html("duration_by_cause.html")
print("duration_by_cause.html")

duration_by_cause.html


In [ ]:
# Create a 'YearMonth' column for aggregation
df_valid['YearMonth'] = df_valid['OUTAGE.START'].dt.to_period('M').astype(str)
outages_over_time = df_valid.groupby('YearMonth').size().reset_index(name='count')

fig1 = px.line(
    outages_over_time,
    x='YearMonth',
    y='count',
    title='Number of Power Outages Over Time',
    labels={'YearMonth': 'Month', 'count': 'Number of Outages'},
    markers=True
)
fig1.update_layout(xaxis_title="Month", yaxis_title="Number of Outages")

# Save the graph to an HTML file to embed in your website
fig1.write_html("outages_over_time.html")
print("Saved to project/outages_over_time.html")

Saved to project/outages_over_time.html


In [ ]:
print("\nGenerating Graph 4: Outage Distribution by Climate Region...")
outages_by_region = df_valid['CLIMATE.REGION'].value_counts().reset_index()
outages_by_region.columns = ['CLIMATE.REGION', 'count']

fig4 = px.treemap(
    outages_by_region,
    path=[px.Constant("All Regions"), 'CLIMATE.REGION'],
    values='count',
    title='Proportion of Power Outages by Climate Region',
    color_discrete_sequence=px.colors.qualitative.Pastel
)
fig4.update_layout(margin = dict(t=50, l=25, r=25, b=25))

# Save the graph to an HTML file
fig4.write_html("outages_by_region.html")
print("Saved to project/outages_by_region.html")


Generating Graph 4: Outage Distribution by Climate Region...
Saved to project/outages_by_region.html


In [ ]:

import folium


map = folium.Map(location=[37.8, -96], zoom_start=3)


for _, row in df[df["U.S._STATE"].notna() & df["CUSTOMERS.AFFECTED"].notna()].iterrows():
    state = row["U.S._STATE"]
    if state in state_coords:
        affected = row["CUSTOMERS.AFFECTED"]
        
        radius = min(30, max(4, affected**0.3 / 3)) 
        color = 'red' if affected > 500_000 else 'orange' if affected > 100_000 else 'green'
        
        folium.CircleMarker(
            location=state_coords[state],
            radius=radius,
            color=color,
            fill=True,
            fill_opacity=0.7,
            popup=f"{state}<br>Cause: {row['CAUSE.CATEGORY']}<br>Affected: {int(affected):,}"
        ).add_to(map)


with open("outages_map.html", "w") as f:
    f.write(map._repr_html_())

In [69]:
pivot_df = df_valid.pivot_table(
    values="DURATION_HOURS",
    index="CLIMATE.REGION",
    columns="CLIMATE.CATEGORY",
    aggfunc="mean"
).round(1)

pivot_html = pivot_df.to_html(classes="styled-table", border=0)
with open("pivot_table.html", "w") as f:
    f.write(pivot_html)

## Step 3: Framing a Prediction Problem
Predict the cause of a major power outage.

In this project, we aim to predict the cause of a major power outage in the United States using relevant environmental, regional, and temporal data available at the time of the outage. The goal is to assist utility companies and policy-makers in proactively identifying risk factors and preparing mitigation strategies.
	
	
	
•	Problem Type: This is a multiclass classification task, as the target variable contains more than two distinct categories.
•	Response Variable: CAUSE.CATEGORY — this column represents the high-level cause of the power outage (e.g., intentional attack, equipment failure, public appeal, etc.). We chose this variable because understanding the cause of an outage has practical value for prevention and planning.
•	Evaluation Metric: We use the macro-averaged F1-score to evaluate model performance. This metric is more appropriate than accuracy because our dataset is imbalanced, with some cause categories appearing much less frequently than others. Macro F1 equally weights each class, ensuring that rare but important categories are not ignored.



## Step 4: Baseline Model

In [67]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score

# Step 1: Select relevant columns
features = [
    'YEAR', 'MONTH', 'U.S._STATE', 'CLIMATE.REGION',
    'ANOMALY.LEVEL', 'CLIMATE.CATEGORY',
    'CUSTOMERS.AFFECTED', 'DURATION_HOURS'
]
target = 'CAUSE.CATEGORY'

# Drop rows with missing target
df_model = df_valid.dropna(subset=[target])[features + [target]].copy()

# Step 2: Split data
X = df_model[features]
y = df_model[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 3: Define preprocessing
categorical_features = ['U.S._STATE', 'CLIMATE.REGION', 'CLIMATE.CATEGORY']
numerical_features = ['YEAR', 'MONTH', 'ANOMALY.LEVEL', 'CUSTOMERS.AFFECTED', 'DURATION_HOURS']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Step 4: Build pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, solver='lbfgs'))
])

# Step 5: Train
pipeline.fit(X_train, y_train)

# Step 6: Evaluate
y_pred = pipeline.predict(X_test)
macro_f1 = f1_score(y_test, y_pred, average='macro')

print("Macro F1-score (Baseline):", round(macro_f1, 3))
print("\nClassification Report:\n", classification_report(y_test, y_pred, zero_division=0))

Macro F1-score (Baseline): 0.373

Classification Report:
                                precision    recall  f1-score   support

            equipment failure       0.00      0.00      0.00         5
        fuel supply emergency       0.00      0.00      0.00         1
           intentional attack       0.82      0.85      0.84        39
                    islanding       0.60      0.43      0.50         7
                public appeal       0.00      0.00      0.00         4
               severe weather       0.87      0.94      0.90       140
system operability disruption       0.38      0.38      0.38        16

                     accuracy                           0.82       212
                    macro avg       0.38      0.37      0.37       212
                 weighted avg       0.77      0.82      0.79       212



## Step 5: final model

In [65]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.metrics import classification_report


df_valid["YEAR"] = df_valid["YEAR"].astype(int)
df_valid["MONTH"] = pd.to_numeric(df_valid["MONTH"], errors="coerce")
df_valid["CUSTOMERS.AFFECTED"] = pd.to_numeric(df_valid["CUSTOMERS.AFFECTED"], errors="coerce")
df_valid["ANOMALY.LEVEL"] = pd.to_numeric(df_valid["ANOMALY.LEVEL"], errors="coerce")


required_cols = ["YEAR", "MONTH", "CUSTOMERS.AFFECTED", "ANOMALY.LEVEL", "CAUSE.CATEGORY"]
df_clean = df_valid.dropna(subset=required_cols).copy()
df_clean = df_clean[df_clean["CUSTOMERS.AFFECTED"] >= 0]
df_clean = df_clean[df_clean["ANOMALY.LEVEL"].notna()]

def extract_season(X):
    month = X["MONTH"].astype(int)
    return pd.DataFrame(np.select(
        [month.isin([12,1,2]), month.isin([3,4,5]), month.isin([6,7,8]), month.isin([9,10,11])],
        ['Winter', 'Spring', 'Summer', 'Fall'],
        default='Unknown'
    ), columns=["SEASON"])

def safe_log1p(X):
    return np.log1p(np.where(X < 0, 0, X))  # Replace negative values with 0

# === Features and target ===
categorical = ["U.S._STATE", "CLIMATE.CATEGORY"]
numerical = ["YEAR", "MONTH", "CUSTOMERS.AFFECTED", "ANOMALY.LEVEL"]
X = df_clean[categorical + numerical]
y = df_clean["CAUSE.CATEGORY"]

# === Train/Test Split ===
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42, test_size=0.2)

# === Preprocessing ===
preprocessor = ColumnTransformer(transformers=[
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical),
    ("season", Pipeline([
        ("season_gen", FunctionTransformer(extract_season, validate=False)),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]), ["MONTH"]),
    ("num", Pipeline([
        ("log", FunctionTransformer(safe_log1p, validate=False)),
        ("scaler", StandardScaler())
    ]), numerical)
])

# === Final Model Pipeline ===
pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

# === Grid Search ===
param_grid = {
    "classifier__n_estimators": [50, 100],
    "classifier__max_depth": [5, 10, 15],
    "classifier__min_samples_split": [2, 5]
}
grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring="f1_macro", n_jobs=-1)
grid_search.fit(X_train, y_train)

# === Evaluation ===
y_pred = grid_search.predict(X_test)
print("Best Parameters:", grid_search.best_params_)
print("\nClassification Report:\n", classification_report(y_test, y_pred, zero_division=0))

Best Parameters: {'classifier__max_depth': 15, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 50}

Classification Report:
                                precision    recall  f1-score   support

            equipment failure       0.00      0.00      0.00         5
        fuel supply emergency       0.00      0.00      0.00         1
           intentional attack       0.93      0.95      0.94        39
                    islanding       0.80      0.57      0.67         7
                public appeal       0.00      0.00      0.00         4
               severe weather       0.87      0.98      0.92       140
system operability disruption       0.40      0.25      0.31        16

                     accuracy                           0.86       212
                    macro avg       0.43      0.39      0.40       212
                 weighted avg       0.80      0.86      0.83       212

